In [3]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random

# 1. 데이터 로드: 쿼리 임베딩과 코사인 유사도 행렬
question_embeddings = np.load('../data/train_question_embeddings.npy')
cos_sim_matrix = np.load('../data/cos_sim_matrix.npy')

# 2. cos_sim_matrix에서 무작위 행 선택 후, 해당 행에서 가장 높은 값(top 25)의 인덱스 선택
rand_row = random.randint(0, cos_sim_matrix.shape[0] - 1)
row_values = cos_sim_matrix[rand_row]
selected_indices = np.argsort(row_values)[::-1][:25]

# 3. PCA를 사용해 쿼리 임베딩(고차원)을 3차원으로 축소
pca = PCA(n_components=3)
pca_results = pca.fit_transform(question_embeddings)
pca_df = pd.DataFrame(pca_results, columns=['PC1', 'PC2', 'PC3'])

# 4. 각 포인트에 대해 선택된(유사한) 질문이면 "Similar", 아니면 "Other"로 hover text 지정
pca_df['hover_text'] = ['Similar' if i in selected_indices else 'Other' for i in range(len(pca_df))]

# 5. Plotly를 사용하여 3D 산점도 시각화 (모든 포인트를 동일한 마커로 표현)
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=pca_df['PC1'],
    y=pca_df['PC2'],
    z=pca_df['PC3'],
    mode='markers',
    marker=dict(size=3, color='gray'),  # 모든 포인트를 동일한 스타일로 표현
    text=pca_df['hover_text'],
    hoverinfo='text'
))

# 6. 레이아웃 업데이트 및 HTML 파일로 저장
fig.update_layout(
    title='PCA 3D Visualization of Embeddings',
    template='plotly_white',
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.write_html("embed_visualization.html")

In [3]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import ast

embeddings = np.load('../data/train_answer_embeddings.npy')
test_rag_df = pd.read_csv('../data/test_rag_result_05.csv', encoding='utf-8-sig')
train_df = pd.read_csv('../data/train_04.csv', encoding='utf-8-sig')

test_id = random.randint(0, len(test_rag_df))
rag_results_ids = test_rag_df.loc[test_id, 'rag_ids']
selected_ids = ast.literal_eval(rag_results_ids)

# 1. PCA를 사용하여 200차원 임베딩을 3차원으로 축소
pca = PCA(n_components=3)
pca_results = pca.fit_transform(embeddings)
pca_df = pd.DataFrame(pca_results, columns=['PC1', 'PC2', 'PC3'])

# 2. train_df에 해당하는 부분만 선택합니다.
#    (train_df의 index가 전체 데이터의 index와 동일하다고 가정)
pca_df_sub = pca_df.loc[train_df.index].reset_index(drop=True)
train_df_sub = train_df.reset_index(drop=True)

# 3. train_df의 'id' 컬럼을 기준으로 마스크 생성 (선택된 데이터와 선택되지 않은 데이터)
selected_mask = train_df_sub['id'].isin(selected_ids)
non_selected_mask = ~train_df_sub['id'].isin(selected_ids)

# 4. Plotly를 이용한 3D 산점도 생성
fig = go.Figure()

# (a) 선택되지 않은 포인트: 작고 회색으로 표시
fig.add_trace(go.Scatter3d(
    x=pca_df_sub.loc[non_selected_mask, 'PC1'],
    y=pca_df_sub.loc[non_selected_mask, 'PC2'],
    z=pca_df_sub.loc[non_selected_mask, 'PC3'],
    mode='markers',
    marker=dict(size=3, color='gray'),
    name=f'Non-Selected (n={non_selected_mask.sum()})',
    # text=[f"Q: {q}<br>A: {a}" for q, a in zip(train_df_sub.loc[non_selected_mask, 'structured_description'],
    #                                           train_df_sub.loc[non_selected_mask, 'ground_truth'])],
    hoverinfo='text'
))

# (b) 선택된 포인트: 크고 붉은색으로 강조 표시
fig.add_trace(go.Scatter3d(
    x=pca_df_sub.loc[selected_mask, 'PC1'],
    y=pca_df_sub.loc[selected_mask, 'PC2'],
    z=pca_df_sub.loc[selected_mask, 'PC3'],
    mode='markers',
    marker=dict(size=10, color='red'),
    name=f'Similar Questions (n={selected_mask.sum()})',
    # text=[f"Q: {q}<br>A: {a}" for q, a in zip(train_df_sub.loc[selected_mask, 'structured_description'],
                                            #   train_df_sub.loc[selected_mask, 'ground_truth'])],
    hoverinfo='text'
))

# 5. 레이아웃 업데이트 및 HTML 파일로 저장
fig.update_layout(
    title='PCA 3D Visualization of Embeddings',
    template='plotly_white',
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.write_html("sim_question_embed_visualization.html")

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random

# 1. 데이터 로드: 쿼리 임베딩과 코사인 유사도 행렬
question_embeddings = np.load('../data/train_question_embeddings.npy')
cos_sim_matrix = np.load('../data/cos_sim_matrix.npy')

# 2. cos_sim_matrix에서 무작위 행 선택 후, 해당 행에서 가장 높은 값(top 25)인 인덱스 선택
#    (여기서 자기 자신(self similarity)도 포함될 수 있음)
rand_row = random.randint(0, cos_sim_matrix.shape[0] - 1)
row_values = cos_sim_matrix[rand_row]
# 내림차순 정렬 후 상위 25개 인덱스 추출
selected_indices = np.argsort(row_values)[::-1][:25]

# 3. PCA를 사용해 쿼리 임베딩(고차원)을 3차원으로 축소
pca = PCA(n_components=3)
pca_results = pca.fit_transform(question_embeddings)
pca_df = pd.DataFrame(pca_results, columns=['PC1', 'PC2', 'PC3'])

# 4. 전체 쿼리 데이터에서 selected_indices에 해당하는 마스크 생성
mask = np.zeros(question_embeddings.shape[0], dtype=bool)
mask[selected_indices] = True

# 5. Plotly를 사용하여 3D 산점도 시각화
fig = go.Figure()

# (a) 선택되지 않은 포인트: 작고 회색으로 표시
fig.add_trace(go.Scatter3d(
    x=pca_df.loc[~mask, 'PC1'],
    y=pca_df.loc[~mask, 'PC2'],
    z=pca_df.loc[~mask, 'PC3'],
    mode='markers',
    marker=dict(size=3, color='gray'),
    name=f'Other Questions (n={np.sum(~mask)})',
    hoverinfo='text'
))

# (b) 선택된 포인트: 크고 붉은색으로 표시
fig.add_trace(go.Scatter3d(
    x=pca_df.loc[mask, 'PC1'],
    y=pca_df.loc[mask, 'PC2'],
    z=pca_df.loc[mask, 'PC3'],
    mode='markers',
    marker=dict(size=10, color='red'),
    name=f'Similar Answers (n={np.sum(mask)})',
    hoverinfo='text'
))

# 6. 레이아웃 업데이트 및 HTML 파일로 저장
fig.update_layout(
    title=f'PCA 3D Visualization of Embeddings',
    template='plotly_white',
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.write_html("sim_answer_embed_visualization.html")